# Div Arb Black Scholes

**Use the Black Scholes Pricing Model to purchase underlying and Puts that seem to be underpriced** (we can also look to just purchasing Puts that are underpriced.
    
This looks is a little more nuanced than Vanilla Straight Up Arb. We want to see if Greek Delta is accurate for put options during stock movement after Ex-Dividend Date. If delta is 0 this approach will not work. We calculate our new delta d' that is more accurate by taking into account dividend dates.    
    
Now our goal is to find when 1+d' is "close" to 0, Dividend is "high", and Underlying Volatility is "low"    
Our profit percentage would be Dividend*(1-tax)/(Underlying + Put_price)